In [5]:
def chama_api():
    from confluent_kafka import Producer
    import requests
    import json
    import time
    import pandas as pd
    import os
    
    broker ='kafka-broker:9092'
    topic ='api'
    producer = Producer({
        'bootstrap.servers': broker,
        'group.id': 'my-group',
        'auto.offset.reset': 'earliest',
        'fetch.max.bytes': 1000000000,  # Ajuste conforme necessário
        'max.partition.fetch.bytes': 1000000000  # Ajuste conforme necessário
    })
    
    session = requests.Session()
    headers = {
                    "Content-Type": "application/json",
                    "Accept": "application/json"

                }
    ##AUTENTICACAO

    ###COLOCAR O TOKEN GERADO
    token = 'c0c03033f09c3b6b685d324476a58accc6510b12ca1f9926f1947c78d4f14cce'
    url_base = "http://api.olhovivo.sptrans.com.br/v2.1/"

    r = session.post(url_base+"Login/Autenticar?token="+token, headers=headers)


    ## GET GENERICO
    def _get(path):
        response = session.get(url_base + path)
        data = response.json()
        return data

    ## PEGAR POSICAO
    line = _get("Posicao")
    j = json.dumps(line,ensure_ascii=True)
    data = json.loads(j)
    data_n=json.dumps(data).encode('utf-8')
    
    if not line or line == "null":
        print("API retornou vazio ou null. Nenhum dado enviado.")
        return
    
#divide as mensagens enviadas
    def split_message(data_bytes, chunk_size):
        """
        Divide uma mensagem codificada em bytes em chunks menores.
        :param data_bytes: Mensagem codificada em bytes.
        :param chunk_size: Tamanho máximo de cada chunk em bytes.
        :return: Lista de chunks (bytes).
        """
        chunks = [data_bytes[i:i + chunk_size] for i in range(0, len(data_bytes), chunk_size)]
        return chunks

    chunks = split_message(data_n, chunk_size=750000)    
#envia as mensagens
    for i, chunk in enumerate(chunks):
        producer.produce(topic, value=chunk.strip(), headers={'chunk-id': str(i)})  # Remova .encode('utf-8')
        print(f"Enviado chunk {i} de {len(chunks)}")

    producer.flush()  # Garante que todas as mensagens sejam enviadas